In [1]:
import torch
from torch import nn

In [20]:
class ConvBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        discriminator=False,
        use_activation=True,
        use_batchNorm=True,
        **kwargs
    ):
        super().__init__()
        self.use_activation = use_activation
        self.cnn = nn.Conv2d(in_channels, out_channels, **kwargs, bias=not use_batchNorm)
        self.batchNorm = nn.BatchNorm2d(out_channels) if use_batchNorm else nn.Identity()
        self.activation = (
            nn.LeakyReLU(0.2, inplace=True)
            if discriminator
            else nn.PReLU(num_parameters=out_channels)
        )
    def forward(self, x):
        return self.activation(
            self.batchNorm(self.cnn(x))) if self.use_activation else self.batchNorm(self.cnn(x)
            )

In [9]:
class UpSampleBlock(nn.Module):
    def __init__(self, in_channels, scale_factor):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, in_channels * scale_factor ** 2, 3, 1, 1)
        self.pixel_suffle = nn.PixelShuffle(scale_factor)
        self.activation = nn.PReLU(num_parameters=in_channels)
        
    def forward(self, x):
        return self.activation(self.pixel_suffle(self.conv(x)))

In [10]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.block_1 = ConvBlock(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.block_2 = ConvBlock(
            in_channels,
            in_channels,
            kernel_size=3,
            stride=1,
            padding=1,
            use_activation=False
        )
        
    def forward(self, x):
        output = self.block_1(x)
        output = self.block_2(output)
        return output + x

In [11]:
class Generator(nn.Module):
    def __init__(self, in_channels=3, num_channels=64, num_blocks=16):
        super().__init__()
        self.initial = ConvBlock(
            in_channels, num_channels, kernel_size=9, stride=1, padding=4, use_batchNorm=False
        )
        self.residuals = nn.Sequential(*[ResidualBlock(num_channels) for _ in range(num_blocks)])
        self.convBlock = ConvBlock(
            num_channels, num_channels, kernel_size=3, stride=1, padding=1, use_activation=False
        )
        self.upSamples = nn.Sequential(
            UpSampleBlock(num_channels, 2), UpSampleBlock(num_channels, 2)
        )
        self.final = nn.Conv2d(num_channels, in_channels, kernel_size=9, stride=1, padding=4)
        
    def forward(self, x):
        initial = self.initial(x)
        x = self.residuals(initial)
        x = self.convBlock(x) + initial
        x = self.upSamples(x)
        return torch.tanh(self.final(x))

In [18]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 64, 128, 128, 256, 256, 512, 512]):
        super().__init__()
        blocks = []
        for idx, feature in enumerate(features):
            blocks.append(
            ConvBlock(
                in_channels,
                feature,
                kernel_size=3,
                stride=1 + idx % 2,
                padding=1,
                discriminator=True,
                use_activation=True,
                use_batchNorm=False if idx == 0 else True
            ))
            in_channels = feature
            
        self.blocks = nn.Sequential(*blocks)
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((6, 6)),
            nn.Flatten(),
            nn.Linear(512 * 6 * 6, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 1)
        )
    
    def forward(self, x):
        x = self.blocks(x)
        return self.classifier(x)

In [21]:
def test():
    lr = 24
    with torch.cuda.amp.autocast():
        x = torch.randn((5, 3, lr, lr))
        gen = Generator()
        gen_out = gen(x)
        disc = Discriminator()
        disc_out = disc(gen_out)
        print(gen_out.shape)
        print(disc_out.shape)
test()

torch.Size([5, 3, 96, 96])
torch.Size([5, 1])


In [ ]:
import torchvision.utils as vutils
import cv2

# Load the generator model (assuming you've defined GeneratorResNet)
generator = GeneratorResNet()
generator.load_state_dict(torch.load('saved_models/generator.pth'))
generator.eval()


In [ ]:
# Load and preprocess the LR input image
def load_lr_image(file_path):
    # Load your LR image using OpenCV and return a copy with BGR to RGB conversion
    image = cv2.imread(file_path)
    if image is None:
        print(f"Failed to load image from {file_path}")
        return None
    return image[:, :, ::-1].copy()  # Convert BGR to RGB and make a copy

# Load the LR image
lr_image = load_lr_image(r"D:\COMPUTER ENGINEERING\SEMESTER 7\ML\Project\train_LR\0001.png")

if lr_image is not None:
    # Convert the NumPy ndarray to a PyTorch tensor
    lr_tensor = torch.from_numpy(lr_image).permute(2, 0, 1).float() / 255.0  # Convert to tensor and normalize

    # Define the preprocessing pipeline and apply it to the tensor
    preprocess = transforms.Compose([
        transforms.ToPILImage(),  # Convert to PIL Image
        transforms.Resize((468, 510)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    lr_tensor = preprocess(lr_tensor).unsqueeze(0)

    # Ensure lr_tensor has shape (batch_size, channels, height, width)
    print(lr_tensor.shape)

    # Generate HR image using the generator
    with torch.no_grad():
        hr_tensor = generator(lr_tensor)

    # Convert HR tensor to a NumPy ndarray and save it as an image
    hr_image = hr_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
    hr_image = (hr_image * 255).astype('uint8')
    
    def save_hr_image(file_path, hr_image):
        # Save the HR image using OpenCV
        cv2.imwrite(file_path, cv2.cvtColor(hr_image, cv2.COLOR_RGB2BGR))

    save_hr_image('output_hr_image.jpg', hr_image)